In [6]:
import numpy as np
import plotly.graph_objects as go

In [10]:
# Load the occupancy data from the .npy file
#occupancy_grid = np.load('./object_occupancy.npy')
occupancy_grid = np.load(r'G:\Mixed_Reality\mr_liuxin\nerfstudio\object_occupancy.npy')
#G:\Mixed_Reality\cs\object_occupancy.npy
# Get the dimensions of the grid
grid_size = occupancy_grid.shape[0]

# Generate grid coordinates
x, y, z = np.meshgrid(np.linspace(-1, 1, grid_size), 
                      np.linspace(-1, 1, grid_size), 
                      np.linspace(-1, 1, grid_size),
                      indexing='ij')

# Filter coordinates based on occupancy
occupied_x = x[occupancy_grid]
occupied_y = y[occupancy_grid]
occupied_z = z[occupancy_grid]
print(np.min(occupied_x), np.max(occupied_x))
print(np.min(occupied_y), np.max(occupied_y))
print(np.min(occupied_z), np.max(occupied_z))

# Create a 3D scatter plot
occupancy = go.Scatter3d(x=occupied_x, y=occupied_y, z=occupied_z, mode='markers', marker=dict(size=1))
fig = go.Figure(data=[occupancy])

# Update layout
fig.update_layout(
    title="3D Occupancy Grid",
    scene=dict(
        xaxis=dict(nticks=4, range=[-1, 1]),
        yaxis=dict(nticks=4, range=[-1, 1]),
        zaxis=dict(nticks=4, range=[-1, 1]),
        aspectratio=dict(x=1, y=1, z=1)
    )
)

# Show plot
fig.show()


-0.4173228346456693 0.1811023622047243
-0.19685039370078738 0.2913385826771653
-0.6062992125984252 -0.13385826771653542


In [11]:
# Points for plane fitting
# points = np.array([
#     [ 0.09619215, 0.18419623, -0.27866834], [ 0.09370056, 0.17330916, -0.3003558 ],
#     [ 0.09418262, 0.19857953, -0.27315882], [ 0.08380522, -0.18537372, -0.011829 ],
#     [ 0.07928248, -0.18414381, -0.0044587 ], [ 0.08964445, -0.18383668, -0.01852112],
#     [-0.00375274, -0.14307953, 0.1119528 ], [-0.00676645, -0.13449907, 0.11913031],
#     [-0.00565832, -0.15868491, 0.10510083], [-0.23815173, 0.01478208, 0.02895552],
#     [-0.25153425, 0.03111171, 0.03467682], [-0.23558566, -0.00102766, 0.02254268],
#     [-0.19367798, 0.09395444, 0.11415317], [-0.20212367, 0.1110208, 0.11448489],
#     [-0.20050208, 0.07870765, 0.11527123], [-0.10122801, 0.25870624, 0.15247521],
#     [-0.09928557, 0.26774612, 0.14757934], [-0.11439849, 0.25597614, 0.16148819],
#     [-0.0189445, 0.40594056, -0.05700587], [-0.0194651, 0.4193155, -0.0729074 ],
#     [-0.02585882, 0.40596634, -0.04706441], [ 0.0656293, 0.33922005, -0.17095272],
#     [ 0.06939287, 0.33703187, -0.18619718], [ 0.05656687, 0.35268608, -0.16519219],
#     [-0.01315877, 0.39768305, 0.04828297], [-0.01366454, 0.41207817, 0.03969196],
#     [-0.01842888, 0.39572006, 0.05479426]
# ])
# [[-0.26936138  0.24160808 -0.6258601 ]
#  [-0.39561564  0.23818085 -0.674399  ]
#  [-0.19534665  0.20195109 -0.6442965 ]
#  [-0.34786913  0.04141811 -0.5851798 ]
#  [-0.35503963 -0.00623185 -0.5804028 ]
#  [-0.35726494  0.09039758 -0.5821767 ]
#  [-0.37899184  0.05395722 -0.57630163]
#  [-0.3428531   0.00101632 -0.57511574]
#  [-0.3942355   0.1459786  -0.5959953 ]
#  [-0.15324256 -0.12237218 -0.58626056]
#  [-0.04642405 -0.13374166 -0.64415807]
#  [-0.24181652 -0.08703528 -0.5820075 ]
#  [-0.15036814 -0.11440113 -0.5950389 ]
#  [-0.06570423 -0.06848542 -0.63898534]
#  [-0.2399236  -0.10452168 -0.6172548 ]
#  [-0.14360096 -0.1030031  -0.5890975 ]
#  [-0.11299132 -0.05421607 -0.59716535]
#  [-0.22671413 -0.11387838 -0.6218325 ]
#  [-0.00156535  0.09174292 -0.5964025 ]
#  [-0.06776507  0.17202237 -0.64643544]
#  [-0.00688706  0.02516501 -0.601858  ]
#  [-0.01667343  0.09022323 -0.615941  ]
#  [-0.08223341  0.14563066 -0.6345265 ]
#  [-0.0090977   0.0030711  -0.6552844 ]
#  [-0.0201582   0.09152752 -0.60159343]
#  [-0.05965558  0.16928552 -0.63525593]
#  [-0.03999963  0.04832425 -0.60437226]]
# points = np.array([[-0.26936138, 0.24160808, -0.6258601],
#                    [-0.39561564, 0.23818085, -0.674399],
#                    [-0.19534665, 0.20195109, -0.6442965],
#                    [-0.34786913, 0.04141811, -0.5851798],
#                    [-0.35503963, -0.00623185, -0.5804028],
#                    [-0.35726494, 0.09039758, -0.5821767],
#                    [-0.37899184, 0.05395722, -0.57630163],
#                    [-0.3428531, 0.00101632, -0.57511574],
#                    [-0.3942355, 0.1459786, -0.5959953],
#                    [-0.15324256, -0.12237218, -0.58626056],
#                    [-0.04642405, -0.13374166, -0.64415807],
#                    [-0.24181652, -0.08703528, -0.5820075],
#                    [-0.15036814, -0.11440113, -0.5950389],
#                    [-0.06570423, -0.06848542, -0.63898534],
#                    [-0.2399236, -0.10452168, -0.6172548],
#                    [-0.14360096, -0.1030031, -0.5890975],
#                    [-0.11299132, -0.05421607, -0.59716535],
#                    [-0.22671413, -0.11387838, -0.6218325],
#                    [-0.00156535, 0.09174292, -0.5964025],
#                    [-0.06776507, 0.17202237, -0.64643544],
#                    [-0.00688706, 0.02516501, -0.601858],
#                    [-0.01667343, 0.09022323, -0.615941],
#                    [-0.08223341, 0.14563066, -0.6345265],
#                    [-0.0090977, 0.0030711, -0.6552844],
#                    [-0.0201582, 0.09152752, -0.60159343],
#                    [-0.05965558, 0.16928552, -0.63525593],
#                    [-0.03999963, 0.04832425, -0.60437226]])
#points = np.load(r'G:\Mixed_Reality\cs\nerfstudio\plane_samples.npy')
points = np.load(r'G:\Mixed_Reality\mr_liuxin\nerfstudio\plane_samples.npy')


# Plane equation coefficients
# a, b, c, d = -0.7358716726303101, -0.14783693850040436, -1, -0.013480226509273052
# -0.0742659941315651x + -0.09833580255508423y + -1z + -0.6207793951034546 = 0
# a, b, c, d = -0.0742659941315651, -0.09833580255508423, -1, -0.6207793951034546
#coeffs = np.load(r'G:\Mixed_Reality\cs\nerfstudio\plane_coefficients.npy')
coeffs = np.load(r'G:\Mixed_Reality\mr_liuxin\nerfstudio\plane_coefficients.npy')

a, b, c, d = coeffs

# Bounding box vertices
# bbox_vertices = np.array([
#     [ 0.0114, -0.1916, -0.0216], [-0.5252, -0.1620, -0.3944],
#     [ 0.2723, -0.1808, -0.3962], [-0.2642, -0.1512, -0.7690],
#     [ 0.0221, 0.2614, -0.0010], [-0.5144, 0.2910, -0.3738],
#     [ 0.2830, 0.2722, -0.3756], [-0.2535, 0.3019, -0.7485]
# ])

# aabb = np.array([[-0.4219, -0.2031, -0.6094],
#                  [ 0.1719,  0.2812, -0.1406]])
# -0.19685039370078738 0.2913385826771653
# -0.4173228346456693 0.1811023622047243
# -0.5748031496062992 -0.13385826771653542
# aabb = np.array([[-0.19685039370078738, -0.4173228346456693, -0.5748031496062992],
#                  [0.2913385826771653, 0.1811023622047243, -0.13385826771653542]])
#aabb = np.load('aabb.npy')
aabb = np.load(r'G:\Mixed_Reality\mr_liuxin\nerfstudio\aabb.npy')
min_corner = aabb[0]
max_corner = aabb[1]
# Compute the vertices of the AABB
vertices = np.array([
    [min_corner[0], min_corner[1], min_corner[2]],
    [max_corner[0], min_corner[1], min_corner[2]],
    [max_corner[0], max_corner[1], min_corner[2]],
    [min_corner[0], max_corner[1], min_corner[2]],
    [min_corner[0], min_corner[1], max_corner[2]],
    [max_corner[0], min_corner[1], max_corner[2]],
    [max_corner[0], max_corner[1], max_corner[2]],
    [min_corner[0], max_corner[1], max_corner[2]],
])

# Define the edges of the AABB
edges = [
    [0, 1], [1, 2], [2, 3], [3, 0],  # bottom edges
    [4, 5], [5, 6], [6, 7], [7, 4],  # top edges
    [0, 4], [1, 5], [2, 6], [3, 7],  # side edges
]

# Plot the edges
edge_traces = [
    go.Scatter3d(
        x=[vertices[pair[0], 0], vertices[pair[1], 0]],
        y=[vertices[pair[0], 1], vertices[pair[1], 1]],
        z=[vertices[pair[0], 2], vertices[pair[1], 2]],
        mode='lines',
        line=dict(color='blue', width=2)
    ) for pair in edges
]

# Create a grid for the plane
xx, yy = np.meshgrid(np.linspace(-1, 1, 10), np.linspace(-1, 1, 10))
zz = (-a * xx - b * yy - d) / c

x_values = []
y_values = []
z_values = []
offset = 0
print(points)

""" for point in points:
    if offset % 200 == 0:
        x_values.append(point[0])
        y_values.append(point[1])
        z_values.append(point[2])
    offset += 1

scatter = go.Scatter3d(x=x_values, y=y_values, z=z_values, mode='markers', marker=dict(size=5)) """

scatter = go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], mode='markers', marker=dict(size=5))

# Plot plane
plane = go.Surface(x=xx, y=yy, z=zz, opacity=0.5)

# Plot bounding box
# bbox_edges = [
#     go.Scatter3d(x=bbox_vertices[[i, j], 0], y=bbox_vertices[[i, j], 1], z=bbox_vertices[[i, j], 2], mode='lines', line=dict(color='red'))
#     for i in range(len(bbox_vertices)) for j in range(i+1, len(bbox_vertices))
# ]

# plot aabb (axis aligned bounding box)


# Combine all plots
fig = go.Figure(data=[scatter, plane, occupancy] + edge_traces)

# Update layout
fig.update_layout(
    scene=dict(
        xaxis=dict(nticks=4, range=[-1,1]),
        yaxis=dict(nticks=4, range=[-1,1]),
        zaxis=dict(nticks=4, range=[-1,1]),
        aspectratio=dict(x=1, y=1, z=1)
    )
)

# Show plot
fig.show()


FileNotFoundError: [Errno 2] No such file or directory: 'G:\\Mixed_Reality\\mr_liuxin\\aabb.npy'